In [129]:
# 初期設定
import pathlib
import textwrap

import google.generativeai as genai
import os
from dotenv import load_dotenv

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=API_KEY)

In [130]:
import json
import re

def validate_json_format(json_data, abstract_id=None):
    # 定義されたルールを持つ正しい形式を検証する
    try:
        # JSONデータからバッククォートを削除
        json_data = re.sub(r'^```json|```$', '', json_data.strip())

        # JSONデータが正しいかどうかを解析
        data = json.loads(json_data)

        if not isinstance(data, list):
            return False

        for item in data:
            if not isinstance(item, dict):
                return False

            if 'abstract_id' not in item or 'rules' not in item:
                return False

            if not isinstance(item['abstract_id'], int):
                return False

            if not isinstance(item['rules'], list):
                return False

            for rule in item['rules']:
                if rule not in ['yes', 'no']:
                    return False

        # abstract_idが指定されている場合、そのIDが存在するか確認
        if abstract_id is not None:
            if not any(item['abstract_id'] == abstract_id for item in data):
                return False

        return True

    except json.JSONDecodeError:
        # JSONが正しくない場合、Falseを返す
        return False

# 使用例
json_data = """```json
[
  {
    "abstract_id": 1,
    "rules": ["yes", "no", "no"]
  },
  {
    "abstract_id": 2,
    "rules": ["no", "yes", "yes"]
  },
  {
    "abstract_id": 3,
    "rules": ["yes", "yes", "no"]
  }
]
```"""

is_valid = validate_json_format(json_data, abstract_id=2)
print(is_valid)  # 正しい形式ならTrueを、誤っている場合はFalseを出力


True


In [132]:
# モデルのインスタンスを作成
model = genai.GenerativeModel(
    "gemini-1.5-flash",
    generation_config={"response_mime_type": "application/json"}
)

In [133]:
%%time

# 
Abstract_ID = 1
Abstract = "Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers-8x deeper than VGG nets [40] but still having lower complexity. An ensemble of these residual nets achieves 3.57% error on the ImageNet test set. This result won the 1st place on the ILSVRC 2015 classification task. We also present analysis on CIFAR-10 with 100 and 1000 layers. The depth of representations is of central importance for many visual recognition tasks. Solely due to our extremely deep representations, we obtain a 28% relative improvement on the COCO object detection dataset. Deep residual nets are foundations of our submissions to ILSVRC & COCO 2015 competitions(1), where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation."

# 回答を生成
prompt = """Please answer yes or no if your abstract follows each of the rules in JSON format.

Use this JSON schema:

results = {'abstract_id': int, 'rules': list[str]}
Return: list[results]

Abstract_ID: {Abstract_ID}
Abstract: {Abstract}

Rules:
1. The objectives, methods, results, and conclusions are clearly stated.
2. It avoids the passive voice and uses active expressions.
3. Separating facts from claims.
"""

response = model.generate_content(prompt)

CPU times: user 12.8 ms, sys: 12.9 ms, total: 25.6 ms
Wall time: 1.77 s


In [134]:
# 結果をマークダウンで表示
to_markdown(response.text)

> [{"abstract_id": 1, "rules": ["yes", "no", "yes"]}, {"abstract_id": 2, "rules": ["no", "yes", "no"]}, {"abstract_id": 3, "rules": ["yes", "yes", "yes"]}]


In [135]:
# 結果がJSON形式であるかどうかを検証
json_data = response.text

is_valid = validate_json_format(json_data, abstract_id=1)
print(is_valid)  # 正しい形式ならTrueを、誤っている場合はFalseを出力

True


In [136]:
# 結果をJSONで表示
print(response.text)

[{"abstract_id": 1, "rules": ["yes", "no", "yes"]}, {"abstract_id": 2, "rules": ["no", "yes", "no"]}, {"abstract_id": 3, "rules": ["yes", "yes", "yes"]}]

